## Setup

In [159]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras
import keras_tuner as kt

Load the data.

In [160]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [161]:
# Drop the non-beneficial ID columns, 'EIN'.
application_df = application_df.drop(["EIN"], axis=1)

In [162]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [163]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [164]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [165]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [166]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df["CLASSIFICATION"].value_counts()
test = classification_counts[classification_counts > 1]
test

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [167]:
#create a filter for binns to replace
filter = classification_counts[classification_counts < 1000].index.tolist()

In [168]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = filter

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [169]:
# Count of name column
name_counts = application_df.NAME.value_counts()
name_counts

# Replace name values with counts less than 20 with "Other"
replace_application = list(name_counts[name_counts < 0.005*1260].index)

# Replace in dataframe
for name in replace_application:
    application_df.NAME = application_df.NAME.replace(name,"Other")

# Check to make sure binning was successful
application_df.NAME.value_counts()

NAME
Other                                          20277
PARENT BOOSTER USA INC                          1260
TOPS CLUB INC                                    765
UNITED STATES BOWLING CONGRESS INC               700
WASHINGTON STATE UNIVERSITY                      492
                                               ...  
RCI INC                                            7
MYTEAM TRIUMPH INC                                 7
NATIONAL ASSOCIATION OF CORPORATE DIRECTORS        7
CHRISTIAN MEDICAL & DENTAL SOCIETY                 7
NATIONAL KAPPA KAPPA IOTA INC                      7
Name: count, Length: 316, dtype: int64

In [170]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [171]:
# Reduce the data size for testing
sample_df = application_df.sample(frac=0.3, random_state=78)  # Change frac as needed


# Split our preprocessed data into our features and target arrays
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values
y = application_df["IS_SUCCESSFUL"].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [172]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Define the model

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a *hypermodel*.

You can define a hypermodel through two approaches:

* By using a model builder function
* By subclassing the `HyperModel` class of the Keras Tuner API

You can also use two pre-defined [HyperModel](https://keras.io/api/keras_tuner/hypermodels/) classes - [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) and [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) for computer vision applications.

In this tutorial, you use a model builder function to define the image classification model. The model builder function returns a compiled model and uses hyperparameters you define inline to hypertune the model.

In [173]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(359,)))

  # Tune the number of units in the first Dense layer
  hp_units = hp.Int('units', min_value=32, max_value=350, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))

  # Add a second Dense layer with tunable number of units
  hp_units_2 = hp.Int('units_2', min_value=32, max_value=350, step=32)
  model.add(keras.layers.Dense(units=hp_units_2, activation='relu'))


  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [174]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='hyperparam2',
                     project_name='intro_to_kt')

Reloading Tuner from hyperparam2\intro_to_kt\tuner0.json


The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [175]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [176]:
tuner.search(X_train_scaled, y_train, epochs=25, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [177]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=25, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/25
644/644 [==============================] - 2s 2ms/step - loss: 0.4995 - accuracy: 0.7703 - val_loss: 0.4616 - val_accuracy: 0.7815
Epoch 2/25
644/644 [==============================] - 1s 2ms/step - loss: 0.4399 - accuracy: 0.7887 - val_loss: 0.4496 - val_accuracy: 0.7833
Epoch 3/25
644/644 [==============================] - 1s 2ms/step - loss: 0.4333 - accuracy: 0.7934 - val_loss: 0.4493 - val_accuracy: 0.7833
Epoch 4/25
644/644 [==============================] - 1s 2ms/step - loss: 0.4302 - accuracy: 0.7949 - val_loss: 0.4562 - val_accuracy: 0.7821
Epoch 5/25
644/644 [==============================] - 1s 2ms/step - loss: 0.4285 - accuracy: 0.7943 - val_loss: 0.4547 - val_accuracy: 0.7843
Epoch 6/25
644/644 [==============================] - 1s 2ms/step - loss: 0.4264 - accuracy: 0.7961 - val_loss: 0.4484 - val_accuracy: 0.7879
Epoch 7/25
644/644 [==============================] - 1s 2ms/step - loss: 0.4259 - accuracy: 0.7955 - val_loss: 0.4582 - val_accuracy: 0.7775
Epoch 

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [178]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train_scaled, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/17
644/644 [==============================] - 2s 2ms/step - loss: 0.4981 - accuracy: 0.7716 - val_loss: 0.4920 - val_accuracy: 0.7776
Epoch 2/17
644/644 [==============================] - 1s 2ms/step - loss: 0.4405 - accuracy: 0.7894 - val_loss: 0.4507 - val_accuracy: 0.7821
Epoch 3/17
644/644 [==============================] - 1s 2ms/step - loss: 0.4328 - accuracy: 0.7937 - val_loss: 0.4486 - val_accuracy: 0.7819
Epoch 4/17
644/644 [==============================] - 1s 2ms/step - loss: 0.4305 - accuracy: 0.7922 - val_loss: 0.4511 - val_accuracy: 0.7806
Epoch 5/17
644/644 [==============================] - 1s 2ms/step - loss: 0.4283 - accuracy: 0.7932 - val_loss: 0.4474 - val_accuracy: 0.7866
Epoch 6/17
644/644 [==============================] - 1s 2ms/step - loss: 0.4276 - accuracy: 0.7964 - val_loss: 0.4499 - val_accuracy: 0.7848
Epoch 7/17
644/644 [==============================] - 1s 2ms/step - loss: 0.4269 - accuracy: 0.7968 - val_loss: 0.4502 - val_accuracy: 0.7868
Epoch 

To finish this tutorial, evaluate the hypermodel on the test data.

In [179]:
eval_result = hypermodel.evaluate(X_test_scaled, y_test,)
print("[test loss, test accuracy]:", eval_result)

268/268 [==============================] - 0s 1ms/step - loss: 0.4489 - accuracy: 0.7900
[test loss, test accuracy]: [0.4489021301269531, 0.789970874786377]


The `my_dir/intro_to_kt` directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.